Introduction : 

En cette année olympique 2024, l'excitation autour des Jeux Olympiques atteint son paroxysme. Les athlètes de tout le globe peaufinent leur préparation, apportant avec eux une riche mosaïque de données comprenant le sexe, l'âge, la taille, le poids, le pays, ainsi que des indicateurs socio-économiques et de santé publique tels que les émissions de CO2, le PIB et le taux de mortalité. C'est dans ce contexte que nous lançons un défi de prédiction des médailles olympiques, tirant parti des données historiques pour prédire les performances futures.


Ce starting kit est conçu pour vous introduire à l'élaboration d'un modèle prédictif sophistiqué, visant à estimer le nombre de médailles que chaque pays est susceptible de remporter. Pour y parvenir, nous exploitons un ensemble de données exhaustif, élaboré à partir d'analyses détaillées disponibles sur Kaggle, qui contient des informations sur les athlètes et les conditions de leurs pays respectifs, collectées à partir de diverses compétitions olympiques allant de 1968 à 2016. Les participants sont ainsi invités à plonger dans cet ensemble de données pour concevoir des modèles capables de traduire ces informations diversifiées en prévisions précises pour les Jeux de 2024, établissant un pont entre l'analyse de données et les prouesses sportives sur la scène mondiale.

 
Alors que nous nous préparons à célébrer l'esprit olympique en 2024, ce défi offre une opportunité unique de combiner passion pour le sport et expertise en data science. Bonne chance à tous les participants !
